#  create dummy data and calculate clusters / routes

In [98]:
num_nodes = 50
margin = 5 # time margin for arrival before appointment and stay beyond
working_hours = [8 * 60, 18 * 60] # 8am to 6pm
percentage_of_appointments = 0.1
span_cost_coefficient = 20000 # adjust
slack = 20000 # adjust
penalty_factor = 300000
num_large_clusters = 1
num_small_clusters = 3
lunch_start = 11 * 60  # 12 PM in minutes
lunch_end = 13 * 60  # 2 PM in minutes
lunch_duration = 30

In [99]:
from src.routing import create_nodes_dataframe, custom_clustering, plot_refined_clusters, assign_weekdays_to_clusters, plot_ind_route, plot_all_cluster_routes, create_data_model, plot_all_nodes_with_angles
import pandas as pd
import numpy as np
from collections import Counter
import concurrent.futures

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [100]:
nodes_df, time_matrix = create_nodes_dataframe(num_nodes=num_nodes, min_work_days=5, home_node_id=0, visiting_interval_min=10, visiting_interval_max=30, max_last_visit=20, frac_fixed_app=percentage_of_appointments)

In [101]:
# plot_all_nodes_with_angles(nodes_df)

- in rare cases the below will make trouble because there are two large gaps and a cluster is entirely contained within the second largest leading to size = nan

In [102]:
clusters = custom_clustering(time_matrix.values, nodes_df, num_small_clusters=num_small_clusters, num_large_clusters=num_large_clusters, overnight_factor=1.3, precision=30, verbose=False)

In [103]:
node_to_cluster = {node: cluster for cluster, nodes in clusters.items() for node in nodes}
nodes_df['cluster'] = nodes_df['node_id'].map(node_to_cluster)

- if there are no fixed appointments the below will fail
- requires adjustements to cases where consecutive days > 2

In [104]:
nodes_df = assign_weekdays_to_clusters(nodes_df)

- the below will Fail if there are lunch breaks

In [105]:
nodes_df['visit_day'] = nodes_df['visit_day'].apply(frozenset)

def adjust_opening_hours(row):
    visit_days = row['visit_day']
    opening_hours = row['opening_hours']
    adjusted_hours = {}
    for day in visit_days:
        if day in opening_hours:
            open_time, close_time = opening_hours[day]
            adjusted_open = open_time.hour * 60 + open_time.minute
            adjusted_close = close_time.hour * 60 + close_time.minute
            if len(visit_days) == 2 and max(visit_days) == day: # fail here if lunch break
                adjusted_open += 1440
                adjusted_close += 1440
            adjusted_hours[int(day)] = (adjusted_open, adjusted_close)
    return adjusted_hours

nodes_df['adjusted_opening_hours'] = nodes_df.apply(adjust_opening_hours, axis=1)

In [106]:
def time_to_minutes(t):
    return t.hour * 60 + t.minute + t.second / 60

def adjust_hours(row):
    opening_hours = row['adjusted_opening_hours']
    appointment = row['fixed_appointment']
    if appointment:
        day, start_time, end_time = appointment
        start_minutes = int(time_to_minutes(start_time) - margin)
        end_minutes = int(time_to_minutes(end_time) + margin)

        # Get the highest day key in the adjusted_opening_hours
        max_day_key = max(opening_hours.keys()) if opening_hours else None
        
        # Check if the appointment is on the last (highest) day
        if (day == max_day_key) & (day != 0):
            start_minutes += 1440  # Add 24 hours in minutes
            end_minutes += 1440

        # Adjust opening hours for the appointment day
        if day in opening_hours:
            opening_hours = {day: (start_minutes, end_minutes)}
        else:
            # Add new day if it does not exist
            opening_hours[day] = (start_minutes, end_minutes)
    # make opening_hours a list of tuples
    opening_hours = [[v[0], v[1]] for _, v in opening_hours.items()]
    return opening_hours

# Applying the function
nodes_df['adjusted_opening_hours'] = nodes_df.apply(adjust_hours, axis=1)

In [107]:
nodes_df['cluster_size'] = nodes_df['cluster'].str.split('_').str[0]

In [108]:
def define_clusters(dataframe):
    dataframe['visit_day'] = dataframe['visit_day'].apply(lambda x: tuple(x) if isinstance(x, list) else (x,))
    dataframe['new_clusters'] = dataframe['visit_day'].astype(str).factorize()[0]
    return dataframe

# Apply the function to the DataFrame
clustered_df = define_clusters(nodes_df)

# Convert the DataFrame to the required dictionary format for plotting
refined_clusters = clustered_df.groupby('new_clusters')['node_id'].apply(list).to_dict()

In [109]:
depot_node_data = nodes_df[nodes_df['node_id'] == 0].iloc[0]  # Assuming there is always a row for node 0 in the original DataFrame

result_dfs = {}
for index, group in nodes_df.groupby('visit_day'):
    # Check if depot node is in the current group
    if 0 not in group['node_id'].values:
        # Append depot node data to the group
        group = pd.concat([pd.DataFrame([depot_node_data]), group], ignore_index=True)
    # Now group is guaranteed to include the depot node
    result_dfs[index] = group[['node_id', 'priority', 'adjusted_opening_hours', 'cluster_size', 'visit_day', 'on_site_time']]

- nodes with fixed appointments should have prio 1
- add margin to not arrive before closing - amount of time staying
- would require changes for a 3 day trip
- correct index for printing dropped nodes
- optimize route beyond priority choice

In [111]:
sub_nodes_df = result_dfs[list(result_dfs.keys())[0]]
solutions = {}
route_lists = {}
route_lists_with_travel = {}

def minutes_to_hhmm(minutes_am):
    minutes_am = minutes_am % 1440  # Ensure minutes are within a day
    minutes = minutes_am % 60
    hours = (minutes_am - minutes) // 60  # Use integer division for hours
    return f'{hours:02}:{minutes:02}'

def create_data_model(sub_nodes_df, sub_time_matrix):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = sub_time_matrix
    data['windows'] = sub_nodes_df['adjusted_opening_hours'].tolist()
    data['priorities'] = sub_nodes_df['priority'].tolist()
    data['num_vehicles'] = 1
    data['on_site_time'] = sub_nodes_df['on_site_time'].tolist()
    data['depot'] = 0
    return data

def return_route_and_times(solution, manager, routing, original_node_ids, data):
    """Returns the route along with the start times at each node."""
    index = routing.Start(0)  # Start at the depot.
    route_with_travel = []
    route_without_travel = []
    time_dimension = routing.GetDimensionOrDie('total_time')  # Make sure this matches the dimension name used

    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        original_node_id = original_node_ids[node_index]  # Map back to original node ID
        time_var = time_dimension.CumulVar(index)
        start_time = solution.Min(time_var)
        end_time = start_time + data['on_site_time'][node_index]  # Include on-site time
        route_with_travel.append((original_node_id, start_time, end_time))  # Include end time for better clarity
        route_without_travel.append((original_node_id, start_time))  # Include end time for better clarity
        next_index = solution.Value(routing.NextVar(index))
        
        travel_time = routing.GetArcCostForVehicle(index, next_index, 0) - data['on_site_time'][index]  # Get travel time
        route_with_travel.append(("road", travel_time))
        
        index = next_index

    # Add the final node
    final_node_index = manager.IndexToNode(index)
    final_node_id = original_node_ids[final_node_index]
    final_time_var = time_dimension.CumulVar(index)
    final_start_time = solution.Min(final_time_var)
    final_end_time = final_start_time + data['on_site_time'][final_node_index]
    route_with_travel.append((final_node_id, final_start_time, final_end_time))
    route_without_travel.append((final_node_id, final_start_time))

    return route_with_travel, route_without_travel

def print_route(route_with_times):
    """Prints the route in the desired format."""
    route_str = ""
    for segment in route_with_times:
        if segment[0] == "road":
            route_str += f" - road ({segment[1]}) - "
        else:
            node_id, start_time, end_time = segment
            route_str += f"{node_id} ({minutes_to_hhmm(start_time)}-{minutes_to_hhmm(end_time)})"
    print(route_str)

def solve_vrp(key, sub_nodes_df):
    max_travel_time = 10000 if len(sub_nodes_df['visit_day'].iloc[0]) == 1 else 20000 # adjust
    nodes = sub_nodes_df['node_id'].tolist()
    sub_time_matrix = time_matrix.loc[nodes, nodes].values.tolist()
    sub_time_matrix = [[int(x) for x in row] for row in sub_time_matrix]
    data = create_data_model(sub_nodes_df, sub_time_matrix)
    manager = pywrapcp.RoutingIndexManager(len(data["time_matrix"]), data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node] + data['on_site_time'][from_node]
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    routing.AddDimension(
        transit_callback_index,
        slack,  # upper bound for slack / waiting time
        max_travel_time,  # upper bound for vehicle maximum travel time
        False,  # start cumul to zero
        "total_time"
    )
    time_dimension = routing.GetDimensionOrDie("total_time")
    time_dimension.SetGlobalSpanCostCoefficient(span_cost_coefficient)

    # PENALTY
    for location_index, priority in enumerate(data['priorities']):
        index = manager.NodeToIndex(location_index)
        if index == 0:
            continue
        else:
            routing.AddDisjunction([index], int(round((priority*100)**2*penalty_factor, 0)))

    # OPENING HOURS, LUNCH AND OVERNIGHT BREAKS
    for location_index, windows in enumerate(data['windows']):
        index = manager.NodeToIndex(location_index)
        days = len(windows)
        if days > 1:
            if index < manager.GetNumberOfNodes():
                work_start = working_hours[0]
                work_end = working_hours[1] + 1440
                latest_start = max(work_start, windows[0][0])
                earliest_end = min(work_end, windows[-1][1])
                # print(f'setting the time window from {latest_start} to {earliest_end} for node {location_index}')
                time_dimension.CumulVar(index).SetRange(latest_start, earliest_end)
            for day in range(1, days):
                # print(f'and removing time between days from {working_hours[1] + 1440 * (day-1)} to {working_hours[0] + 1440 * day} for node {location_index}')
                time_dimension.CumulVar(index).RemoveInterval(windows[day-1][1], windows[day][0])
                time_dimension.CumulVar(index).RemoveInterval(working_hours[1] + 1440 * (day-1), working_hours[0] + 1440 * day)
        else:
            if windows[0][0] > 1440:
                # print('work on day 2')
                work_start = working_hours[0] + 1440
                work_end = working_hours[1] + 1440
                # print(f'work starts at {work_start} and ends at {work_end}')
                # print(f'window starts at {windows[0][0]} and ends at {windows[0][1]}')
                day_start = max(windows[0][0], work_start)
                day_end = min(windows[0][1], work_end)
            else:
                day_start = windows[0][0]
                day_end = windows[0][1]
            # print(f'setting the time window from {day_start} to {day_end} for node {location_index}')
            time_dimension.CumulVar(index).SetRange(day_start, day_end)  

    node_visit_transit = {}
    for index in range(routing.Size()):
        node = manager.IndexToNode(index)
        node_visit_transit[index] = data['on_site_time'][node]

    lunch_break_interval = routing.solver().FixedDurationIntervalVar(
        lunch_start, lunch_end, lunch_duration, False, 'lunch_break'
    )

    # Assign the break interval to the single vehicle
    time_dimension.SetBreakIntervalsOfVehicle([lunch_break_interval], 0, node_visit_transit)

    # Instantiate route start and end times to produce feasible times
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(0)))
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(0)))

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # search_parameters.local_search_metaheuristic = (
    #     routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
    search_parameters.time_limit.seconds = 300
    search_parameters.log_search = False

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        dropped = []
        for node in range(routing.Size()):
            if routing.IsStart(node) or routing.IsEnd(node):
                continue
            if solution.Value(routing.NextVar(node)) == node:
                dropped.append(manager.IndexToNode(node))
        if len(dropped) > 0:
            print(f"dropped from {key}: {dropped}")
        return key, return_route_and_times(solution, manager, routing, nodes, data)
        
    else:
        print(f"No solution for key {key}")
        return key, None
    
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_key = {executor.submit(solve_vrp, key, sub_nodes_df): key for key, sub_nodes_df in result_dfs.items()}
    for future in concurrent.futures.as_completed(future_to_key):
        key = future_to_key[future]
        # print(f"Finding solution for key: {key}")
        try:
            key, result = future.result()
            if result:
                route_lists_with_travel[key] = result[0]
                route_lists[key] = result[1]
        except Exception as e:
            print(f"Error for key {key}: {e}")

for key, route in route_lists_with_travel.items():
    print(f"Route for key {key}:")
    print_route(route)

dropped from (frozenset({3.0}),): [6, 7, 8]
dropped from (frozenset({1.0, 2.0}),): [7, 20]
Route for key (frozenset({0.0}),):
0 (08:42-08:42) - road (28) - 9 (09:10-09:50) - road (14) - 44 (10:04-10:45) - road (9) - 40 (10:54-11:28) - road (13) - 8 (12:11-12:51) - road (20) - 6 (13:11-13:45) - road (40) - 35 (14:25-14:55) - road (28) - 23 (15:23-16:14) - road (26) - 47 (16:40-17:34) - road (16) - 7 (17:50-18:37) - road (63) - 0 (19:40-19:40)
Route for key (frozenset({4.0}),):
0 (09:23-09:23) - road (42) - 15 (10:05-10:55) - road (4) - 33 (10:59-11:36) - road (11) - 49 (11:47-12:21) - road (20) - 42 (12:41-13:33) - road (9) - 16 (14:12-14:48) - road (24) - 28 (15:12-16:09) - road (12) - 41 (16:21-17:15) - road (45) - 27 (18:00-18:45) - road (53) - 0 (19:38-19:38)
Route for key (frozenset({3.0}),):
0 (09:01-09:01) - road (11) - 34 (09:12-09:53) - road (20) - 10 (10:13-11:05) - road (7) - 4 (11:12-12:07) - road (25) - 43 (12:32-13:09) - road (25) - 20 (14:04-14:49) - road (8) - 12 (14:57-

# check and visualize results

In [ ]:
route_and_times = route_lists[list(route_lists.keys())[0]]
route_df = pd.DataFrame(route_and_times, columns=['node_id', 'arrival_time'])
merged_df = pd.merge(nodes_df, route_df, on='node_id', how='left')[['adjusted_opening_hours', 'arrival_time', 'node_id']]
merged_df = merged_df[merged_df['node_id'] != 0]

def check_within_ranges(arrival_time, ranges):
    if len(ranges) == 1:
        return ranges[0][0] <= arrival_time <= ranges[0][1]
    elif len(ranges) == 2:
        return (ranges[0][0] <= arrival_time <= ranges[0][1]) or (ranges[1][0] <= arrival_time <= ranges[1][1])
    return False

merged_df['time_check'] = merged_df.apply(
    lambda row: check_within_ranges(row['arrival_time'], row['adjusted_opening_hours']), axis=1
)

if merged_df['time_check'].all():
    print('VIOLATION OF TIME CONSTRAINTS')
    print(merged_df[merged_df['time_check'] == False])

# merged_df[['node_id', 'adjusted_opening_hours', 'arrival_time', 'time_check']]

- print more info (opening hours, fixed appointments, priority, decisions [removing a node, replacing a node to another day, priority, etc])
- use data to fine tune hyperparameters

evtl:
- Store state and if it was possible to find routes for all solutions iteratively add nodes based on node priority and distance to root node (those further away should be included more likely)
- Test Discrete Priority (must be visited this week; must be visited next week; ...)
- Compare routes with and without overnight stays / large clusters

In [116]:
plot_refined_clusters(refined_clusters, nodes_df, home_node_id=0)

In [117]:
plot_all_cluster_routes(route_lists, nodes_df)